In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import wav2vec2, bert, fastspeech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
from malaya_speech.train.model.fastvc.model import Decoder

config = malaya_speech.config.fastspeech_config
dim = 768
config['encoder_hidden_size'] = dim
config['encoder_num_hidden_layers'] = 4
config['encoder_num_attention_heads'] = 12
config['encoder_intermediate_size'] = dim * config['encoder_num_hidden_layers']
config = fastspeech.Config(vocab_size = 1, **config)
encoder = Decoder(config.encoder_self_attention_params)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# class Encoder:
#     def __init__(self, config):
#         self.config = config
#         self.model = None
#     def __call__(self, x, input_mask, training = True):
#         if self.model is None:
#             input_mask = tf.logical_not(input_mask)
#             self.model = bert.BertModel(config = self.config, is_training = training,
#                                   input_ids = x, input_mask = input_mask)
#         return self.model.sequence_output
    
# encoder = Encoder(config = bert.BertConfig())

In [ ]:
cfg = wav2vec2.Wav2Vec2Config()
model = wav2vec2.Model(cfg, encoder)

In [ ]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [ ]:
r, num_vars, curr_temp = model(X, padding_mask = X_len)
r

In [ ]:
logits = r['x']
logits = tf.transpose(logits, [2, 1, 0])
logits = tf.reshape(logits, (-1, tf.shape(logits)[-1]))
target = tf.zeros(shape = (tf.shape(r['x'])[1] * tf.shape(r['x'])[2]), dtype = tf.int32)
logits, target

In [ ]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target, logits = logits)
loss = tf.reduce_sum(loss)

In [ ]:
extra_losses = []

if 'prob_perplexity' in r:
    extra_losses.append(
        (num_vars - r["prob_perplexity"])
        / num_vars
    )

if "features_pen" in r:
    extra_losses.append(r["features_pen"])

In [ ]:
sample_size = tf.cast(tf.shape(target)[0], tf.float32)

In [ ]:
loss_weights = [0.1, 10]
for p, coef in zip(extra_losses, loss_weights):
    if coef != 0 and p is not None:
        p = coef * p * sample_size
        loss += p

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

In [ ]:
y = np.random.normal(size = (16000 * 10))

In [ ]:
%%time

o = sess.run([r, logits, target, loss], feed_dict = {X: [y, y], X_len: [len(y), len(y)]})

In [ ]:
o[-1]

In [23]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [24]:
!ls -lh test
!rm -rf test

total 746808
-rw-r--r--  1 huseinzolkepli  staff    77B Apr 15 20:59 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   344M Apr 15 20:59 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   3.7K Apr 15 20:59 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    13M Apr 15 20:59 model.ckpt.meta
